# 🤖 AgentPro Quick Start Guide

This notebook will walk you through how to set up and use [AgentPro](https://github.com/traversaal-ai/AgentPro) — a production-ready open-source agent framework built by [Traversaal.ai](https://traversaal.ai) for building powerful, modular, and multi-functional AI agents.

### What is AgentPro?
AgentPro lets you build intelligent agents that can:
- Use language models (like OpenAI’s GPT) as reasoning engines
- Solve real-world tasks such as research, automation, and knowledge retrieval
- Scale up with custom tools, memory, and orchestration features

Whether you're a developer, researcher, or AI enthusiast — this guide will help you:
- Build and integrate your own tools with AgentPro


## Step 1: Clone AgentPro and Install Dependencies

To get started with **AgentPro**, begin by cloning the official GitHub repository and installing its dependencies.

In [1]:
!pip install git+https://github.com/traversaal-ai/AgentPro.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 5.2 MB/s eta 0:00:00


## Step 2: Set Your API Key

To use OpenAI models with AgentPro, you’ll need an API key from OpenAI. Follow these steps:

1. Go to the [OpenAI API platform](https://platform.openai.com/)
2. Log in or create an account
3. Click "Create new secret key"
4. Copy the generated key and paste it into the notebook like this:

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "<openai_api_key>"

## Step 3: Create a Custom Tool
AgentPro is designed to be extensible — you can easily define your own tools for domain-specific tasks.

Below is an example of a **custom tool** that queries the Hugging Face Hub and returns the **most downloaded model** for a given task:

In [3]:
from huggingface_hub import list_models

# Define the task you're interested in
task_name = "text-classification"

# Get the most downloaded model for the specified task
models = list_models(filter=task_name, sort="downloads", direction=-1)
top_model = next(iter(models))

# Print the model ID
print(top_model.id)

distilbert/distilbert-base-uncased-finetuned-sst-2-english


## Step 4: Define your tool using AgentPro Tool class

In [6]:
# Imports
import os
from agentpro import ReactAgent
from agentpro.tools import Tool
from huggingface_hub import list_models
from typing import Any
from agentpro import create_model

class MostModelTool(Tool):
    name: str = "Most Downloaded Model Finder" # Human-readable tool name
    description: str = "Finds the most downloaded model for a given task on Hugging Face." # Brief explanation of tool for agent
    action_type: str = "find_top_model" # Use lowercase letters with underscores for agent; avoid spaces, digits and special characters
    input_format: str = "Task name as a string. Example: 'text-classification'" # Expected input dtype with example

    def run(self, input_text: Any) -> str:
        task_name = input_text.strip()
        models = list_models(filter=task_name, sort="downloads", direction=-1)
        top_model = next(models)
        return top_model.id

## Step 5: Pass tool to AgentPro

In [7]:
# Instantiate your tools and ReactAgent
tools = [MostModelTool()]

# Create a model with OpenAI
model = create_model(provider="openai", model_name="gpt-4o", api_key=os.getenv("OPENAI_API_KEY", None))

myagent = ReactAgent(model=model, tools=tools)

# Run
query = "Can you give me the name of the model that has the most downloads in the 'text-classification' task on the Hugging Face Hub?"
response = myagent.run(query)
# Can you give me the name of the model that has the most downloads in the 'text-classification' task on the Hugging Face Hub?
# Find the most popular model used for 'object-detection' on Hugging Face.

print(f"\nFinal Answer: {response.final_answer}")

================================================== Iteration 1 
✅  [Debug] Sending System Prompt (with history) to LLM:
You are an AI assistant that follows the ReAct (Reasoning + Acting) pattern.
        
Your goal is to help users by breaking down complex tasks into a series of thought-out steps and actions.

You have access to these tools: find_top_model

Tool: Most Downloaded Model Finder
Description: Finds the most downloaded model for a given task on Hugging Face.
Action Type: find_top_model
Input Format: Task name as a string. Example: 'text-classification'


Your task is to:
1. Think about what action is required — Thought.
2. Take an appropriate action — Action.
3. Repeat Thought/Action as needed until you find the final answer.

### Format (Choose only one per response)

Option 1 — When action is needed:
Thought: Your reasoning about action and observation.
Action: {"action_type": "<action_type>", "input": <input_data>}

Option 2 — When you're confident in the final response: